<a href="https://colab.research.google.com/github/feranzie/RNN-script-generator/blob/main/RNN_PLAY_GENERATOR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [79]:
%tensorflow_version 2.x  # this line is not required unless you are in a notebook
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [80]:
#download txt data
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [81]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


# Encoding & Decoding

In [82]:
vocab = sorted(set(text))
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
#encode characters with numbers
def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [83]:
# lets look at how part of our text is encoded
print("Text:", text[:13])
print("Encoded:", text_to_int(text[:13]))

Text: First Citizen
Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [84]:
#decodes encoded numbers to text
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
    #turn array of characters to a word
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

First Citizen


# Training Example

In [85]:
seq_length = 100  # length of sequence for a training example
#+1 is here since we have a sequence of 100 for both inputs and outputs and were dropping last letter of a word e.g Hell and output is dropping first letter and predicting last letterello 
#hence if input is 100 to make room for lastletter or character been added to end of ouput we add +1
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
#slices text into streams of characters in this case 1.1 million characters *check len(text)
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [86]:
char_dataset


<TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int64, name=None)>

In [87]:
#splits characters into batches of 101 words and drop remaining
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [88]:
def split_input_target(chunk):  # for the example: hello
    input_text = chunk[:-1]  # hell
    target_text = chunk[1:]  # ello
    return input_text, target_text  # hell, ello
#perform above fuction to delete last letter from input text and delete first letter from target_text
dataset = sequences.map(split_input_target)  # we use map to apply the above function to every entry

In [89]:
#each input set and outtput set is 100 from sequence we set earlier
#were trying to see how what input sets and output sets look like for 3 sets
for x, y in dataset.take(3):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))




EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXAMPLE

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


EXAMPLE

INPUT
now Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us k

OUTPUT
ow Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us ki


In [90]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

# Build model


In [91]:
#build model
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE,EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (64, None, 256)           16640     
                                                                 
 lstm_3 (LSTM)               (64, None, 1024)          5246976   
                                                                 
 dense_3 (Dense)             (64, None, 65)            66625     
                                                                 
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


# Testing prediction output  for specific input

In [95]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)  # ask our model for a prediction on our first batch of training data (64 entries)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")  # print out the output shape

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [96]:
# we can see that the predicition is an array of 64 arrays, one for each entry in the batch
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[-3.16558918e-03 -6.82592997e-03 -4.69417358e-03 ...  3.06074088e-03
    6.51968701e-04 -6.04538899e-03]
  [ 7.34083820e-04 -7.60643161e-04 -1.59248640e-03 ...  6.23427238e-03
    3.18496046e-03 -2.88132206e-03]
  [-6.68469770e-03 -7.74360774e-03  3.55181540e-03 ...  2.64929864e-03
    1.09571293e-02 -1.01597989e-02]
  ...
  [-9.32968222e-03 -1.41425384e-03  3.39564076e-03 ...  5.50788827e-04
   -1.49654057e-02  5.11864061e-03]
  [-9.18117911e-03 -4.84004058e-03  1.97905255e-03 ... -5.58256265e-03
   -1.48872407e-02  6.86002523e-03]
  [-8.27649515e-03 -6.00500079e-03  7.25320482e-04 ... -8.56997911e-03
   -1.22804455e-02  1.25783877e-02]]

 [[-3.19438823e-03 -6.18128851e-03  1.96089735e-03 ... -5.61899913e-04
   -7.46111711e-03 -1.40070484e-03]
  [-5.42336842e-04 -7.63581926e-03  3.46666202e-05 ... -4.19596676e-03
   -9.73248389e-03 -7.17152515e-03]
  [-3.43859755e-03 -6.91180257e-03  5.09509305e-03 ... -5.77628566e-03
   -7.75142480e-03  5.98224578e-05]
  ...
  [-3.880

In [97]:
# lets examine one prediction
pred = example_batch_predictions[0]
print(len(pred))
print(pred)
# notice this is a 2d array of length 100, where each interior array is the prediction for the next character at each time step

100
tf.Tensor(
[[-0.00316559 -0.00682593 -0.00469417 ...  0.00306074  0.00065197
  -0.00604539]
 [ 0.00073408 -0.00076064 -0.00159249 ...  0.00623427  0.00318496
  -0.00288132]
 [-0.0066847  -0.00774361  0.00355182 ...  0.0026493   0.01095713
  -0.0101598 ]
 ...
 [-0.00932968 -0.00141425  0.00339564 ...  0.00055079 -0.01496541
   0.00511864]
 [-0.00918118 -0.00484004  0.00197905 ... -0.00558256 -0.01488724
   0.00686003]
 [-0.0082765  -0.006005    0.00072532 ... -0.00856998 -0.01228045
   0.01257839]], shape=(100, 65), dtype=float32)


In [98]:
# and finally well look at a prediction at the first timestep
time_pred = pred[0]
print(len(time_pred))
print(time_pred)
# and of course its 65 values representing the probabillity of each character occuring next

65
tf.Tensor(
[-0.00316559 -0.00682593 -0.00469417 -0.00083912 -0.00335781  0.00330693
 -0.00065697 -0.00144205  0.00375655  0.00554464 -0.00660874  0.00403293
 -0.00569704  0.0037281  -0.00622446 -0.00160995  0.00307979  0.00299281
  0.00780211  0.00630144 -0.00601348  0.0014077   0.00386529  0.00623795
 -0.00085527 -0.00577341  0.0027416   0.00282463 -0.00897866 -0.00014697
  0.00472694 -0.00993719 -0.00158849 -0.00157537 -0.00238053  0.00077021
 -0.00511616 -0.0058402   0.00986343 -0.00222157 -0.00394132 -0.0036018
  0.00264283 -0.00316038  0.00414871 -0.01009093  0.00243318  0.00065907
 -0.0020988   0.00276029 -0.00387131 -0.00047051  0.0018606  -0.00248282
 -0.00055158 -0.0043747   0.00059106  0.00245873  0.00226678 -0.0001444
 -0.00337273 -0.00247454  0.00306074  0.00065197 -0.00604539], shape=(65,), dtype=float32)


the output has a shape of (64,100,65)
meaning 64 batches, 100 output characters,65 predictions of what the output text is supposed to be they are then sampled and one of the 65 is chosen as each character of the 100 output characters.

In [99]:
# If we want to determine the predicted character we need to sample the output distribution (pick a value based on probabillity)
sampled_indices = tf.random.categorical(pred, num_samples=1)

# now we can reshape that array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars  # and this is what the model predicted for training sequence 1

"xCICZncWRguslDi&E'hT.QpGj\naDIbaTIFYXBz:yoxeDX'syElmp3WvNt o:Aotcyj,q,v,-D\nNhAYqV,juzg&N3-&th;vAFLpTs"

# Create Loss function

In [100]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

# Compile model
At this point we can think of our problem as a classification problem where the model predicts the probabillity of each unique letter coming next.

In [101]:
model.compile(optimizer='adam', loss=loss)

# Creating Checkpoints
Now we are going to setup and configure our model to save checkpoinst as it trains. This will allow us to load our model from a checkpoint and continue training it.



In [102]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

# Training model

In [105]:
history = model.fit(data, epochs=10, callbacks=[checkpoint_callback])

Epoch 1/10
172/172 [==============================] - 14s 73ms/step - loss: 1.1647
Epoch 2/10
172/172 [==============================] - 14s 75ms/step - loss: 1.1274
Epoch 3/10
172/172 [==============================] - 14s 73ms/step - loss: 1.0888
Epoch 4/10
172/172 [==============================] - 14s 72ms/step - loss: 1.0498
Epoch 5/10
172/172 [==============================] - 13s 72ms/step - loss: 1.0093
Epoch 6/10
172/172 [==============================] - 13s 71ms/step - loss: 0.9679
Epoch 7/10
172/172 [==============================] - 14s 73ms/step - loss: 0.9268
Epoch 8/10
172/172 [==============================] - 14s 73ms/step - loss: 0.8866
Epoch 9/10
172/172 [==============================] - 14s 72ms/step - loss: 0.8457
Epoch 10/10
172/172 [==============================] - 13s 71ms/step - loss: 0.8085


# Loading the Model
We'll rebuild the model from a checkpoint using a batch_size of 1 so that we can feed one peice of text to the model and have it make a prediction.

In [106]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

Once the model is finished training, we can find the lastest checkpoint that stores the models weights using the following line.

In [107]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

We can load any checkpoint we want by specifying the exact file to load.

In [111]:
checkpoint_num = 10
model.load_weights(tf.train.load_checkpoint("./training_checkpoints/ckpt_" + str(checkpoint_num)))
model.build(tf.TensorShape([1, None]))

AttributeError: ignored


# Generating Text
Now we can use the lovely function provided by tensorflow to generate some text using any starting string we'd like.

In [109]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 800

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
    
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [110]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))

Type a starting string: love is war
love is wary of shine terrors.

WARWICK:
O hou! Valt, one work, parting in
the subdues of your name, I beseech you.
Come, Camillo, an what little sort to Bohemia,
Since I am limit to my father was mine own
A thousand duch, which thou threat'st our country pray.
There's some showers, love, with hand and how much
her luck war, in the hollow grow of a man.

BIONUED:
There's no more said when that's made fair rash. This is an irother's gald,
That instige thy thousand King of Henry, revenge,
Too dear account the men, she is lefthe way.
I'll know the mins have gatest.

ARIEL:
My lord!

KING HENRY VI:
And let us hear her speak a'l goodly to him,
And rest to prison, of the queen and his blisson. I dare not offence
That woe requite awhile,
It shall be soleth time have lent her schill;
Let's marry her old abov
